# Projeto 1 - Ciência dos Dados

Nome: Maria Carolina Porto 

Nome: Thomas Kassabian

Nome: Kaique Tinto

Nome: Eduardo Candeias

___
Carregando algumas bibliotecas:

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\othom\OneDrive - Insper - Institudo de Ensino e Pesquisa\insper\Disciplinas\2\CDados\Projetos\CDADOS_Projeto1


Carregando a base de dados com as mensagens dos seus arquivos:

In [4]:
train = pd.read_csv('dados_treino_QUARTETO_Thomas.csv')

In [5]:
test = pd.read_csv('dados_teste_QUARTETO_Thomas.csv')

___
## Classificador automático


Esse classificador tem por objetivo ensinar um Boot a classificar corretamente os reviews de clientes de uma companhia aérea. Para isso, utilizamos um dataFrame "Train" para treiná-lo, e um dataFrame distinto "Test" para testá-lo.

As possíveis classificações são:
* Detractor: Clientes que expressaram insatisfação ou descontentamento significativo com a companhia aérea;
* Promoter: Clientes extremamente satisfeitos e leais à companhia aérea;
* Passive: Clientes que não expressaram satisfação ou insatisfação com a companhia aérea.

___
## Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

### Preparação dos dados

#### Limpeza e filtragem da base de dados

- Padronização dos dados: removendo letras maiúsculas, pontuações e acentos.
- Filtragem dos dados: removendo stopwords e palavras com duas letras.

In [6]:
# Função que limpa um texto

import re

def cleanup(text):    
    # Retira a pontuação
    punctuation = '[´"!-.:?;$''()]'
    pattern = re.compile(punctuation)
    
    # Retira as stopwords (palavras indiferentes para o cálculo da probabilidade)
    stopwords = ['the','to', 'and', 'was', 'i', 'a', 'in', 'on', 'of', 'with','for', 'flight', 'we']
    words = r'\b(?:' + '|'.join(map(re.escape, stopwords)) + r')\b'
    
    # Retira palavras de duas letras
    limpa_duas_letras = r'\b\w{1,2}\b'
    

    # Aplica as alterações e retorna o texto limpo
    text = re.sub(pattern, '', text)
    text = re.sub(words, '', text, flags=re.IGNORECASE)
    text = re.sub(limpa_duas_letras, '', text)

    return text

#### Gerando a base de dados

In [7]:
# Função que recebe um dataframe, limpa e transforma em lista
def transf_lista(dataframe):
    texto = " "

    for linha in dataframe['Review']:
        texto += cleanup(linha)
    
    lista = texto.lower().split()
    return lista

In [8]:
# Criando pd.Series para cada classificação distinta e para o total
# pdSeires com repetição de palavras

serie_train = pd.Series(transf_lista(train))
serie_Detractor = pd.Series(transf_lista(train.loc[(train.NPS == 'Detractor'), :]))
serie_Promoter = pd.Series(transf_lista(train.loc[(train.NPS == 'Promoter'), :]))
serie_Passive = pd.Series(transf_lista(train.loc[(train.NPS == 'Passive'), :]))

#### Contagem dos dados (frequência absoluta e relativa)

In [9]:
# Criando uma tabela com a contagem de cada palavra em cada classificação
# pdSeries sem repetição de palavras

tabela_train = serie_train.value_counts()
tabela_Detractor = serie_Detractor.value_counts()
tabela_Promoter = serie_Promoter.value_counts()
tabela_Passive = serie_Passive.value_counts()

In [10]:
# Criando uma tabela com a porcentagem relativa de cada palavra em cada classificação

tabela_Detractor_relativo = serie_Detractor.value_counts(True)
tabela_Promoter_relativo = serie_Promoter.value_counts(True)
tabela_Passive_relativo = serie_Passive.value_counts(True)

In [11]:
# Função que calcula a probabilidade de uma palavra estar em um NPS (classificação) específico

# Recebe: palavra -> palavra que se deseja calcular a probabilidade;
#         tab_NPS -> tabela de um NPS específico;
#         len_serie_NPS -> quantidade de palavras totais (com repetição) em um NPS específico.

def calc_prob(palavra, tab_NPS, len_serie_NPS):
    prob = 1
    # Se a palavra estiver na tabela do NPS específico
    if palavra in tab_NPS:
        prob = ((tab_NPS[palavra] + 1)/(len_serie_NPS+ len(tabela_train)))*1e3
        
    # Se a palavra for inédita
    else:   
        prob = (1/(len_serie_NPS + len(tabela_train)))*1e3

    return prob

----
### Treinamento

#### Cálculo das probabilidades

In [12]:
# Probabilidade da palavra ser de cada NPS específico em relação ao total

P_D = len(serie_Detractor)/len(serie_train)
P_Pa = len(serie_Passive)/len(serie_train)
P_Pr = len(serie_Promoter)/len(serie_train)

In [13]:
# Classifica a frase em detractor, promoter, passive

classif = []

# For que percorre todos os reviews do dataFrame
for frase in train.Review:

    #Limpar a frase
    frase_limpa = cleanup(frase)
    frase_limpa = frase_limpa.lower().split()
    
    P_frase_dado_D = 1
    P_frase_dado_Pa = 1 
    P_frase_dado_Pr = 1

    # For que percorre cada palavra das frases
    for palavra in frase_limpa:

        #Probabilidade da frase ser de cada NPS específico
        P_frase_dado_D *= calc_prob(palavra, tabela_Detractor, len(serie_Detractor))
        P_frase_dado_Pa *= calc_prob(palavra, tabela_Passive, len(serie_Passive))
        P_frase_dado_Pr *= calc_prob(palavra, tabela_Promoter, len(serie_Promoter))

    #Probabilidade de cada NPS específico para a frase
    P_D_dado_frase = (P_frase_dado_D * P_D)
    P_Pa_dado_frase = (P_frase_dado_Pa * P_Pa)
    P_Pr_dado_frase = (P_frase_dado_Pr * P_Pr)

    # Armazena as probabilidades, compara e adiciona a maior classif na lista 
    prob = [P_D_dado_frase, P_Pa_dado_frase, P_Pr_dado_frase]

    if max(prob) == prob[0]:
        classif.append("Detractor")
    elif max(prob) == prob[1]:
        classif.append("Passive")
    elif max(prob) == prob[2]:
        classif.append("Promoter")

#### Verificando os resultados

Comparação do NPS com a classificação fornecida pelo programa

In [14]:
# Cria uma nova coluna com as classificações do Boot
train['Boot'] = classif

# Assertividade
assertividade = len(train.loc[(train.NPS == train.Boot), :])/len(train)
print('Asssertividade: {:.2f}%'.format(assertividade*100))

# DF com os reviews classificados corretamente
certo = train.loc[train.NPS == train.Boot, :]
certo

Asssertividade: 85.67%


,Review,NPS,Boot
0,Bangkok to Phuket round trip. The lounge at th...,Promoter,Promoter
1,A real mixed bag with Air New Zealand from Auc...,Passive,Passive
2,Second in the queue in business class check-...,Detractor,Detractor
3,London Heathrow to Riyadh return. Pleasant f...,Promoter,Promoter
4,Hong Kong to Bangkok. Check-in at the transf...,Promoter,Promoter
...,...,...,...
4193,Great Christmas holiday in Spain starting wit...,Promoter,Promoter
4194,Gothenburg to London. Flights were on time. S...,Detractor,Detractor
4196,After 3 hours in the plane waiting for repairs...,Detractor,Detractor
4198,My mother left on AI 43 MAA-DEL connecting AI ...,Promoter,Promoter


Crosstab (NPS x Boot)

In [15]:
pd.crosstab(train.NPS, train.Boot)

Boot,Detractor,Passive,Promoter
NPS,,,
Detractor,1751,110,104
Passive,109,412,165
Promoter,113,1,1435


___
### Testes

#### Cálculo das probabilidades

In [16]:
classif_test = []

for frase in test.Review:

    #Limpar a frase
    frase_limpa = cleanup(frase)
    frase_limpa = frase_limpa.lower().split()
    
    P_frase_dado_D = 1
    P_frase_dado_Pa = 1 
    P_frase_dado_Pr = 1

    # For que percorre cada palavra das frases
    for palavra in frase_limpa:

        #Probabilidade da frase ser de cada NPS específico
        P_frase_dado_D *= calc_prob(palavra, tabela_Detractor, len(serie_Detractor))
        P_frase_dado_Pa *= calc_prob(palavra, tabela_Passive, len(serie_Passive))
        P_frase_dado_Pr *= calc_prob(palavra, tabela_Promoter, len(serie_Promoter))

    #Probabilidade de cada NPS específico para a frase
    P_D_dado_frase = (P_frase_dado_D * P_D)
    P_Pa_dado_frase = (P_frase_dado_Pa * P_Pa)
    P_Pr_dado_frase = (P_frase_dado_Pr * P_Pr)

    # Armazena as probabilidades, compara e adiciona a maior classif na lista 
    prob = [P_D_dado_frase, P_Pa_dado_frase, P_Pr_dado_frase]

    if max(prob) == prob[0]:
        classif_test.append("Detractor")
    elif max(prob) == prob[1]:
        classif_test.append("Passive")
    elif max(prob) == prob[2]:
        classif_test.append("Promoter")

#### Verificando os resultados
Comparação do NPS com a classificação fornecida pelo programa

Crosstab (NPS x Boot)

In [18]:
pd.crosstab(test.NPS, test.Boot)

Boot,Detractor,Passive,Promoter
NPS,,,
Detractor,719,86,45
Passive,86,39,145
Promoter,83,23,574


Porcentagens

In [19]:
total_promoter = len(test.loc[test.NPS == 'Promoter', :])
total_passive = len(test.loc[test.NPS == 'Passive', :])
total_detractor = len(test.loc[test.NPS == 'Detractor', :])

# Classificados corretamente
certo_promoter = len(test.loc[(test.NPS == 'Promoter') & (test.Boot == 'Promoter'), :])
certo_passive = len(test.loc[(test.NPS == 'Passive') & (test.Boot == 'Passive'), :])
certo_detractor = len(test.loc[(test.NPS == 'Detractor') & (test.Boot == 'Detractor'), :])

# Classificados incorretamente
errado_promoter = 1 - certo_promoter/total_promoter
errado_passive = 1 - certo_passive/total_passive
errado_detractor = 1 - certo_detractor/total_detractor

# Acurácia
acuracia_test = len(test.loc[(test.NPS == test.Boot), :])/len(test)

# Print
print('Asssertividade: {:.2f}%\n'.format(acuracia_test*100))
print(f'Promoter\n Verdadeiros: {certo_promoter/total_promoter*100:.2f}%\n Falsos: {errado_promoter*100:.2f}%\n')
print(f'Passive\n Verdadeiros: {certo_passive/total_passive*100:.2f}%\n Falsos: {errado_passive*100:.2f}%\n')
print(f'Detractor\n Verdadeiros: {certo_detractor/total_detractor*100:.2f}%\n Falsos: {errado_detractor*100:.2f}%\n')

Asssertividade: 74.00%

Promoter
 Verdadeiros: 84.41%
 Falsos: 15.59%

Passive
 Verdadeiros: 14.44%
 Falsos: 85.56%

Detractor
 Verdadeiros: 84.59%
 Falsos: 15.41%



___
### Concluindo

**1. Comparativo entre os percentuais:**

O classificador tende a identificar corretamente reviews classificadas como "Promoter" e "Detractor". Entretanto, reviews referentes a classificação "Passive" possuem elevado índice de erro. 


**2. Dupla negação e sarcasmo:**

Reviews com dupla negação, que nós humanos identificamos como algo positivo, o classificador identificará como duas vezes algo negativo, aumentando a probabilidade de ser classificada como "Detractor", quando provavelmente não é. Por ooutro lado, reviews com sarcasmo, que nós humanos identificamos como algo negativo, o classificador identificará como algo positivo, aumentando a probabilidade de ser classificada como "Promoter", quando provavelmente não é. 

Ou seja, duplas negações e frases sarcásticas aumentam as chances de classificação incorreta.

**3. Plano de expansão:**

O nosso projeto deve continuar sendo financiado pois a classificação das reviews dos clientes da companhia aérea auxilia positivamente a empresa, uma vez que eles conseguem identificar, por meio de dados concretos, se os clientes estão contentes ou não com os serviços oferecidos. Além disso, é possível identificar quais são os principais pontos de contentamento e descontentamento, os quais, somado a um plano de ação da empresa, possibilitará melhorias na companhia, buscando melhorar os serviços oferecidos e a experiência do usuário. 

Além disso, caso a companhia desejasse enviar reviews positivas e negativas para setores distintos, o classificador removeria a necessidade de que um funcionário lesse cada review e a classificasse, poupando tempo e recursos financeiros da empresa, uma vez que um classificador faria automaticamente tal divisão. 

**4. Diferentes cenários para Naïve Bayes fora do contexto do projeto:**

* Detectar Fake News;

* Detectar mensagens ofensivas/ inapropriadas para, possivelmente, retirar da internet;

**5. Possíveis melhorias para o projeto:**

* Aplicar os métodos stemming e lemmatization na limpeza das frases.

Stemming: Analisa cada palavra individualmente e a reduz para o seu radical (não será necessariamente gramaticamente correta);

Lemmatization: Analisa cada palavra individualmente e a reduz para o seu radical (será necessariamente gramaticamente correta);

Ambos os métodos podem melhorar a nossa ánalise ao juntar palavras difernetes mas com o radical semelhante, alterando a probabilidade final e, portanto, a classificação final (material de pesquisa em "Referências").

* Aplicar metodos de redes neurais, para podermos considerar a semantica e ordem das palavras, com o intuito de aumentar a assertividade do código.



___
### Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* OK IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens. Ex: stemming, lemmatization, stopwords
* OK CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* OK EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* OK PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* OK SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* OK FEZ o item Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

[Lemmatization vs. stemming: quando usar cada uma?](https://www.alura.com.br/artigos/lemmatization-vs-stemming-quando-usar-cada-uma) **Diferenças entre Lemmatization e stemming, e como aplicar cada uma**

In [ ]:
# O modelo probabilistico para o calculo e classificação de texto, se baseia na probabilidade de uma palavra estar em um NPS específico. Fizemos o uso do metodo de Naive Bayes para calcular a probabilidade de uma palavra estar em um NPS específico. Com o intuito de minimizar os erros e maximizar a porcentagem de acertividade, aplicamos a suavização de Laplace dentro do código 